<a href="https://colab.research.google.com/github/Mrezakhodashenas/PhD-NN/blob/output-spikes/Noisy_intermediate_layers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%reset -f

In [2]:
!pip install torch

In [3]:
!pip install wandb --upgrade

In [4]:
!pip install pytorch-lightning

In [5]:
!pip install snntorch

In [6]:
!pip install torchsummary

In [7]:
!pip install wandb onnx -Uq

## set seeds for PyTorch and Numpy to ensure reproducibility:

In [8]:
import torch
import random
import numpy as np

# Set seeds for Python, Numpy, and Torch for reproducibility
# Ensure deterministic behavior
torch.backends.cudnn.deterministic = True
random.seed(hash("setting random seeds") % 2**32 - 1)
np.random.seed(hash("improves reproducibility") % 2**32 - 1)
torch.manual_seed(hash("by removing stochasticity") % 2**32 - 1)
torch.cuda.manual_seed_all(hash("so runs are repeatable") % 2**32 - 1)


# seed = 42
# random.seed(seed)
# np.random.seed(seed)
# torch.manual_seed(seed)
# torch.cuda.manual_seed_all(seed)

# Additional steps if you're using GPU
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False


In [9]:
# imports
import pickle
import matplotlib.animation as animation
from scipy.integrate import simps
import torch
torch.cuda.empty_cache()
import os, sys, time, datetime, json, random
import snntorch as snn
from snntorch import spikeplot as splt
from snntorch import spikegen
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
import os
import torch.nn.functional as F
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from torchvision import utils as utls
from snntorch import utils
from snntorch import surrogate
import numpy as np
import math
from sklearn.metrics import auc
from torchsummary import summary
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
import torch.nn as nn
from scipy.stats import entropy
from scipy.special import kl_div
from torch.autograd import Variable
from sklearn.metrics import accuracy_score
# from skopt import gp_minimize
# from bayes_opt import BayesianOptimization
import wandb
from tqdm.auto import tqdm
import matplotlib.ticker as mtick



# import spikeflow as snn

In [10]:
# fe7dfc869c7793392e4f225d9d64f615e2fd70ec
wandb.login()

wandb: Currently logged in as: mreza-khodashenas (neuroscience_ai). Use `wandb login --relogin` to force relogin


True

In [11]:
wandb.init(project="Noisy_StdDev0_1_Intermediate_Layers")      # Set the project where this run will be logged

config = dict(
    std_dev = 0.1,
    subset_fraction = 1,     # the percentage of the data used
    epochs=50,

    batch_size=250,
    learning_rate=0.0001,
    input_size = 32,
    kernels=[32, 64, 128],
    threshold_Real = 0.5,
    threshold_Recon = 0.5,

    alpha=0.9,
    # beta_syn=0.0001,
    beta_syn=0.9,
    beta =0.9,
    num_steps=5,
    latent_dim = 32, #dimension of latent layer (how compressed we want the information)
    thresh=1,    #spiking threshold (lower = more spikes are let through)

    lr=0.0001,
    betas=(0.9, 0.999),
    weight_decay=0.001,

    dataset="MNIST",
    architecture="SAE(L_Syn_Syn_L)")

## set seeds for PyTorch and Numpy to ensure reproducibility:

In [12]:
start_time = datetime.datetime.now()

# /////////////////////# Building the Autoencoder
#-------------------DataLoaders.  using the MNIST dataset

# dataloader arguments
batch_size = 250
data_path='/data/mnist'

dtype = torch.float
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")


# /////////////////////////////////# Define a transform
input_size = 32 # resizing the original MNIST from 28 to 32

transform = transforms.Compose([
            transforms.Resize((input_size, input_size)),
            transforms.Grayscale(),
            transforms.ToTensor(),
            transforms.Normalize((0,), (1,))])

#------------------------------------------- Load MNIST
# Training data
train_dataset = datasets.MNIST(root='dataset/', train=True, transform=transform, download=True)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

# Testing data
test_dataset = datasets.MNIST(root='dataset/', train=False, transform=transform, download=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)



In [13]:
# creating directories where we can save the original and reconstructed images for training and testing:
# create training/ and testing/ folders in the chosen path
if not os.path.isdir('figures/training'):
    os.makedirs('figures/training')
if not os.path.isdir('figures/binarytraining'):
    os.makedirs('figures/binarytraining')

if not os.path.isdir('figures/testing'):
    os.makedirs('figures/testing')
if not os.path.isdir('figures/binarytesting'):
    os.makedirs('figures/binarytesting')


if not os.path.isdir('Saved_Trained_Checkpoints/'):
    os.makedirs('Saved_Trained_Checkpoints/')

if not os.path.isdir('Output_Spikes/'):
    os.makedirs('Output_Spikes/')

if not os.path.isdir('Enc_syn_Spikes/'):
    os.makedirs('Enc_syn_Spikes/')


if not os.path.isdir('Intermediate_Lyrs/'):
    os.makedirs('Intermediate_Lyrs/')



In [14]:
import torch

if torch.cuda.is_available():
    device = torch.device("cuda")
    print(f"Using {torch.cuda.get_device_name()} ({device})")
else:
    device = torch.device("cpu")
    print("Using CPU")


Using Tesla T4 (cuda)


In [15]:
class SAE(nn.Module):
    def __init__(self):
        super().__init__()


        # encoder_layers = [
        #     ('conv1', nn.Conv2d(1, 32, 3, padding=1, stride=2)),  # Conv Layer 1
        #     ('batchnorm1', nn.BatchNorm2d(32)),
        #     ('leaky1', snn.Leaky(beta=beta, spike_grad=spike_grad, init_hidden=True, output=True, threshold=thresh)),
        #     ('conv2', nn.Conv2d(32, 64, 3, padding=1, stride=2)),  # Conv Layer 2
        #     ('batchnorm2', nn.BatchNorm2d(64)),
        #     ('synaptic1', snn.Synaptic(alpha=alpha, beta=beta_syn, spike_grad=spike_grad, init_hidden=True, output=True, threshold=thresh)),  # SNN TORCH LIF NEURON
        #     ('conv3', nn.Conv2d(64, 128, 3, padding=1, stride=2)),  # Conv Layer 3
        #     ('batchnorm3', nn.BatchNorm2d(128)),
        #     ('synaptic2', snn.Synaptic(alpha=alpha, beta=beta_syn, spike_grad=spike_grad, init_hidden=True, output=True, threshold=thresh)),  # SNN TORCH LIF NEURON
        #     ('flatten', nn.Flatten(start_dim=1, end_dim=3)),  # Flatten convolutional output
        #     ('linear', nn.Linear(128 * 4 * 4, latent_dim)),  # Fully connected linear layer
        #     ('leaky2', snn.Leaky(beta=beta, spike_grad=spike_grad, init_hidden=True, output=True, threshold=thresh))
        # ]


        # Encoder
        self.encoder = nn.Sequential(
                            nn.Conv2d(1, 32, 3,padding = 1,stride=2), # Conv Layer 1
                            nn.BatchNorm2d(32),
                            snn.Leaky(beta=beta, spike_grad=spike_grad, init_hidden=True,threshold=thresh),
                            nn.Conv2d(32, 64, 3,padding = 1,stride=2), # Conv Layer 2
                            nn.BatchNorm2d(64),
                            snn.Synaptic(alpha=alpha, beta=beta_syn, spike_grad=spike_grad, init_hidden=True, threshold=thresh), #SNN TORCH LIF NEURON
                            nn.Conv2d(64, 128, 3,padding = 1,stride=2), # Conv Layer 3
                            nn.BatchNorm2d(128),
                            snn.Synaptic(alpha=alpha, beta=beta_syn, spike_grad=spike_grad, init_hidden=True,threshold=thresh), #SNN TORCH LIF NEURON
                            nn.Flatten(start_dim = 1, end_dim = 3), #Flatten convolutional output
                            nn.Linear(128*4*4, latent_dim), # Fully connected linear layer
                            snn.Leaky(beta=beta, spike_grad=spike_grad, init_hidden=True, output=True,threshold=thresh)
                            )


        self.latent_dim = latent_dim #dimensions of the encoded z-space data
        self.linearNet= nn.Sequential(
                                      nn.Linear(latent_dim,128*4*4),
                                      snn.Leaky(beta=beta, spike_grad=spike_grad, init_hidden=True, output=True,threshold=thresh))

        # Decoder:
        self.decoder = nn.Sequential(
                            nn.Unflatten(1,(128,4,4)), #Unflatten data from 1 dim to tensor of 128 x 4 x 4
                            snn.Leaky(beta=beta, spike_grad=spike_grad, init_hidden=True,threshold=thresh),
                            nn.ConvTranspose2d(128, 64, 3,padding = 1,stride=(2,2),output_padding=1),
                            nn.BatchNorm2d(64),
                            snn.Synaptic(alpha=alpha, beta=beta_syn, spike_grad=spike_grad, init_hidden=True,threshold=thresh),
                            nn.ConvTranspose2d(64, 32, 3,padding = 1,stride=(2,2),output_padding=1),
                            nn.BatchNorm2d(32),
                            snn.Synaptic(alpha=alpha, beta=beta_syn, spike_grad=spike_grad, init_hidden=True,threshold=thresh),
                            nn.ConvTranspose2d(32, 1, 3,padding = 1,stride=(2,2),output_padding=1),
                            snn.Leaky(beta=beta, spike_grad=spike_grad, init_hidden=True,output=True,threshold=20000) #so membrane can be trained
                            )
    def forward(self, x):
        utils.reset(self.encoder) #need to reset the hidden states of LIF
        utils.reset(self.decoder)
        utils.reset(self.linearNet)

    #-----------------------------encode
        spk_mem=[];
        spk_rec=[];
        spk_rec_syn=[];
        encoder_mem=[];
        spk_rec_dec=[];
        spk_mem_dec=[];
        enc5_rec = [];


     #------------------------------ intermediate layers

        # for step in range(num_steps):
        #     enc5 = self.encoder[5](x)             #  enc5 shape: torch.Size([250, 1, 32, 32])
        #     enc5_rec.append(enc5)
        # enc5_rec = torch.stack(enc5_rec, dim=2)            #   enc5_rec size: torch.Size([250, 1, 5, 32, 32])
        # Enc_syn_1 = enc5_rec[:, :, -1]                      # #   torch.Size([250, 1, 32, 32])

     #------------------------------ encode
        for step in range(num_steps):
            spk_x, mem_x = self.encoder(x)              # spk_x size: ([250, 32])  ,   mem_x size: ([250, 32])  , x.shape : torch.Size([250, 1, 32, 32])
            spk_rec.append(spk_x)
            spk_mem.append(mem_x)

        spk_rec=torch.stack(spk_rec,dim=2) # stack spikes in second tensor dimension # ----------------spk_rec in torch.stack(spk_rec,dim=2):  torch.Size([250, 32, 5])
        spk_mem=torch.stack(spk_mem,dim=2) # stack membranes in second tensor dimension # ----------------spk_mem in torch.stack(spk_mem,dim=2):  torch.Size([250, 32, 5])
        out_en = spk_rec[...,step]

        # print("out_en= spk_rec[...,step]:-----------" , spk_rec[...,step].size()) # spk_rec[...,step]:----------- torch.Size([250, 32])       input of the latent and then decoder

     #------------------------------decode
        spk_mem2=[];
        spk_rec2=[];
        decoded_x=[];
        spk_x_dec=[];
        mem_x_dec=[];
        for step in range(num_steps): #for t in time                           #        from decoder: ([250, 1, 32, 32])
            x_recon, x_mem_recon = self.decode(spk_rec[...,step])
            spk_rec2.append(x_recon)
            spk_mem2.append(x_mem_recon)

        spk_rec2=torch.stack(spk_rec2,dim=4)
        spk_mem2=torch.stack(spk_mem2,dim=4)

        out = spk_mem2[:,:,:,:,-1]

        self.out_en = out_en
        self.out = out

        return out, out_en
        # return out, Enc_syn_1

    def encode(self,x):
      spk_latent_x, mem_latent_x = self.encoder(x)
      return spk_latent_x, mem_latent_x


    def decode(self,x):
        spk_x, mem_x = self.linearNet(x) #convert latent dimension back to total size of features in encoder final layer
        spk_x2, mem_x2 = self.decoder(spk_x)
        return spk_x2, mem_x2



In [16]:
import numpy as np

def calculate_pixel_accuracy(real_img, x_recon):
    # Flatten the tensors and convert them to NumPy arrays
    real_img_np = real_img.flatten().cpu().numpy()
    x_recon_np = x_recon.flatten().detach().cpu().numpy()

    # Calculate pixel-wise similarity
    pixel_accuracy = 1 - np.mean(np.abs(x_recon_np - real_img_np))

    return pixel_accuracy


In [17]:
# Training and Testing
# using MSE loss to compare the reconstructed image (x_recon) with the original image (real_img)
from torchvision.utils import save_image

spike_recordings = []
train_ber_rec = []
test_ber_rec = []
threshold_Real = 0.5
threshold_Recon = 0.5

def train(network, trainloader, opti, epoch):
    # Tell wandb to watch what the model gets up to: gradients, weights, and more!
    wandb.watch(network, log="all", log_freq=10)
    network=network.train()
    train_loss_hist=torch.zeros((1), dtype=dtype, device=device)
    train_avg_loss_rec=[]

    for batch_idx, (real_img, labels) in enumerate(trainloader):
        opti.zero_grad()
        real_img = real_img.to(device)
        labels = labels.to(device)

      # -------------Add Gaussian noise to the input images
        real_img = real_img + torch.randn_like(real_img) * std_dev


        out, out_en = network(real_img)  # Pass data into network and return reconstructed image and spk_rec
        x_recon, out = network(real_img)  # Pass data into network and return reconstructed image and spk_rec.  #        x_recon size torch.Size([250, 1, 32, 32]) ,  #        out size torch.Size([250, 32])
        #Calculate loss
        loss_val = torch.zeros((1), dtype=dtype, device=device)
        for step in range(num_steps):
          loss_val += F.mse_loss(x_recon, real_img)                  #.view(1, -1)
        train_loss_hist += (loss_val.item())/num_steps
        avg_loss=train_loss_hist.mean()

        # # ---------------------------- Calculate Bit Error Rate (BER)
        real_img_binary = (real_img > threshold_Real).float()
        x_recon_binary = (x_recon > threshold_Recon).float()
        bit_errors = torch.sum(torch.abs(real_img_binary - x_recon_binary))
        total_pixels = real_img_binary.numel()  # Total number of pixels in the images
        bit_error_rate = bit_errors.item() / total_pixels
        train_ber_rec.append(bit_error_rate)  # Append BER to the list

        # ---------------
        loss_val.backward()
        opti.step()
        train_loss_rec.append(loss_val.item())
        # ----------------



        # Save binary images
        Error_bin = (torch.abs(x_recon_binary - real_img_binary))

        if batch_idx == len(trainloader)-1:
          if epoch in [0, 25, 49]:
            utls.save_image(real_img_binary, f'figures/binarytraining/ep{epoch}_inputs_binary.png')
            utls.save_image(x_recon_binary, f'figures/binarytraining/ep{epoch}_recon_binary.png')
            utls.save_image(Error_bin, f'figures/binarytraining/ep{epoch}_Error_bin.png')

        # Logging for wandb
        if (batch_idx + 1) % 10 == 0:
          wandb.log({"Epoch": epoch, "Train_Batch": batch_idx, "Train_Loss": loss_val.item(), "Train_BER": bit_error_rate})


        print(f'Train[{epoch}/{max_epoch}][{batch_idx}/{len(trainloader)}] Loss: {loss_val.item()}, ' f'BER : {bit_error_rate}')


        #Save reconstructed images every at the end of the epoch
        if batch_idx == len(trainloader)-1:
          if epoch in [0, 10, 25, 49]:
            utls.save_image((real_img+1)/2, f'figures/training/epoch{epoch}_finalbatch_inputs.png')
            utls.save_image((x_recon+1)/2, f'figures/training/epoch{epoch}_finalbatch_recon.png')
            train_auc = auc(np.arange(len(train_loss_rec)), train_loss_rec)

    epoch_loss = sum(train_loss_rec) / len(train_loss_rec)
    print(f'Epoch [{epoch}/{max_epoch}] Train_Avg_loss(epoch): {epoch_loss:.5f}')
    print('-------------------------------------')
    wandb.log({"Epoch": epoch, "Train_AvgLoss(Epoch)": epoch_loss})

    # return loss_val, train_loss_rec, train_auc , out, out_en  #, spk_rec_batches#, train_avg_loss_rec, #avg_loss #, train_loss_hist
    return loss_val, train_loss_rec, epoch_loss , out, out_en      # train_accuracy, epoch_accuracy, noisy_out_en -----------------added

    # return loss_val, train_loss_rec, train_auc   #


# For Testing, not doing backpropagate, therefore no gradients are required and we use torch.no_grad():
#Testing Loop
def test(network, testloader, opti, epoch):
    network=network.eval()
    test_loss_hist=[]
    test_avg_loss_rec=[]
    test_avg_loss_hist = []

    spk_rec_test = []
    with torch.no_grad(): #no gradient this time
        for batch_idx, (real_img, labels) in enumerate(testloader):
            real_img = real_img.to(device)#
            labels = labels.to(device)

            out, out_en = network(real_img)  # Pass data into network and return reconstructed image and spk_rec
            x_recon , out = network(real_img)  # Pass data into network and return reconstructed image and spk_rec
            # average Loss:
            loss_val = torch.zeros((1), dtype=dtype, device=device)
            for step in range(num_steps):
              loss_val += F.mse_loss(x_recon, real_img)
            avg_loss=loss_val/num_steps
            test_loss_hist.append(loss_val.item())

            real_img_binary = (real_img > threshold_Real).float()
            x_recon_binary = (x_recon > threshold_Recon).float()
            bit_errors = torch.sum(torch.abs(real_img_binary - x_recon_binary))
            total_pixels = real_img_binary.numel()  # Total number of pixels in the images
            bit_error_rate = bit_errors.item() / total_pixels
            test_ber_rec.append(bit_error_rate)  # Append BER to the list

            # Save binary images
            Error_bin = (torch.abs(x_recon_binary - real_img_binary))

            if batch_idx == len(testloader)-1:
              if epoch in [0, 25, 49]:
                save_image(real_img_binary, f'figures/binarytesting/ep{epoch}_inputs_binary.png')
                save_image(x_recon_binary, f'figures/binarytesting/ep{epoch}_recon_binary.png')
                save_image(Error_bin, f'figures/binarytesting/ep{epoch}_Error_bin.png')


        # Logging for wandb
            if (batch_idx + 1) % 10 == 0:
              wandb.log({"Epoch": epoch, "Test_Batch": batch_idx, "Test_Loss": loss_val.item(), "Test_BER": bit_error_rate})

            # -------------------------------------------------------------------------------------------------

            print(f'Test[{epoch}/{max_epoch}][{batch_idx}/{len(testloader)}]  Loss: {loss_val.item()}, '  f'BER (test): {bit_error_rate}')

            test_loss_rec.append(loss_val.item())

            if batch_idx == len(testloader)-1:
              if epoch in [0, 10, 25, 49]:
                utls.save_image((real_img+1)/2, f'figures/testing/epoch{epoch}_finalbatch_inputs.png')
                utls.save_image((real_img+(torch.randn_like(real_img) * std_dev)+1)/2, f'figures/testing/(real_img+noise)_epoch{epoch}_finalbatch_inputs.png')
                utls.save_image((x_recon+1)/2, f'figures/testing/epoch{epoch}_finalbatch_recons.png')
                test_auc = auc(np.arange(len(test_loss_rec)), test_loss_rec)

    epoch_loss_test = sum(test_loss_hist) / len(test_loss_hist)
    print(f'Test_Epoch [{epoch}/{max_epoch}] Test_Avg_loss(epoch): {epoch_loss_test:.5f}')

    wandb.log({"Test_Epoch": epoch, "Test_AvgLoss(Epoch)": epoch_loss_test})

    # return loss_val, test_loss_rec, test_auc, out, out_en                     # -------------------------------------------------ADDED
    return loss_val, test_loss_rec, epoch_loss_test, out, out_en     # ------------------------------------- noisy_out_en  , test_accuracy ------------------ADDED

    # return loss_val, test_loss_rec, test_auc  #

for batch_spikes in spike_recordings:
    print(batch_spikes)


In [18]:
input_size = 32 #resize of mnist data (optional)

#setup GPU
dtype = torch.float
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

# neuron and simulation parameters
spike_grad = surrogate.atan(alpha=2.0)  # alternate surrogate gradient fast_sigmoid(slope=25)

train_loss_rec = []
test_loss_rec = []
train_loss_record = []
test_loss_record = []
train_avg_loss_rec=[]
test_avg_loss_rec=[]

  # Synaptic current and membrane potential decay exponentially with rates of alpha and beta
alpha=0.9
# beta_syn=0.0001
beta_syn=0.9
beta =0.9
std_dev=0.1


num_steps=5
latent_dim = 32 #dimension of latent layer (how compressed we want the information)
thresh=1    #spiking threshold (lower = more spikes are let through)
epochs=50
# epochs=5
max_epoch=epochs

  #Define Network and optimizer
net=SAE()
net = net.to(device)
optimizer = torch.optim.AdamW(net.parameters(),
                            lr=0.0001,
                            betas=(0.9, 0.999),
                            weight_decay=0.001)



activation = {}
# def get_activation(name):
#     def hook(model, input, output):
#         activation[name] = output.detach()
#     return hook


def get_activation(name):
    def hook(model, input, output):
        if isinstance(output, tuple):
            activation[name] = [out.detach() for out in output]
        else:
            activation[name] = output.detach()
    return hook


# net.encoder[5].register_forward_hook(get_activation('encoder[5]'))


## for saving after each epoch:

In [ ]:
checkpoint_path = "Saved_Trained_Checkpoints/"
Output_Spikes = "Output_Spikes/"
Enc_syn_Spikes = "Enc_syn_Spikes/"
Intermediate_Lyrs = "Intermediate_Lyrs/"
epoch_activations_list = [] # Create a list to store activations for each epoch
epoch_activations = {}



# # Run training and testing
# for e in range(epochs):
#     train_loss = train(net, train_loader, optimizer, e)
#     train_avg_loss_rec.append(sum(train_loss_rec) / len(train_loader))

#     test_loss = test(net, test_loader, optimizer, e)
#     test_avg_loss_rec.append(sum(test_loss_rec) / (len(test_loader)))


#     #----------Save the model every 10 epochs
#     if (e + 1) % 10 == 0:
#         model_path = checkpoint_path + f"model_epoch_{e + 1}.pt"
#         torch.save(net.state_dict(), model_path)

#     # # ---------------------------------------------- Add hooks for specific layers

#     hook_layers = [net.encoder[2], net.encoder[5], net.encoder[8], net.encoder[11], net.decoder[1], net.decoder[4], net.decoder[7], net.decoder[9]]  # You can choose the layers you want to capture activations from
#     hook_names = ["Enc_Lk1","Enc_syn1", "Enc_syn2", "Enc_Lk2", "Dec_Lk1", "Dec_syn1", "Dec_syn2", "Dec_Lk2"]  # Names for the captured activations
#     hooks = []
#     for i, layer in enumerate(hook_layers):
#       hook_fn = get_activation(hook_names[i])
#       hooks.append(layer.register_forward_hook(hook_fn))

#     epoch_activations[e] = {}
#     for i, name in enumerate(hook_names):
#       epoch_activations[e][name] = activation[name]

#     # Check if the current epoch is a multiple of 10
#     if (e + 1) % 10 == 0:
#       # Save the epoch_activations dictionary to a file
#       activations_path = Intermediate_Lyrs + f"Intermediate_Lyrs_epoch_{e + 1}.pkl"
#       with open(activations_path, 'wb') as file:
#         pickle.dump(epoch_activations, file)



# ///////////////////////////////////

# Define hook_layers and hook_names
hook_layers = [net.encoder[2], net.encoder[5], net.encoder[8], net.encoder[11], net.decoder[1], net.decoder[4], net.decoder[7], net.decoder[9]]
hook_names = ["Enc_Lk1", "Enc_syn1", "Enc_syn2", "Enc_Lk2", "Dec_Lk1", "Dec_syn1", "Dec_syn2", "Dec_Lk2"]

# Create an empty dictionary to store activations
epoch_activations = {}

# Register hooks for capturing activations
hooks = []
for i, layer in enumerate(hook_layers):
    hook_fn = get_activation(hook_names[i])
    hooks.append(layer.register_forward_hook(hook_fn))


# -----------------------------------------------------------
import os

# Run training and testing
for e in range(epochs):
    train_loss = train(net, train_loader, optimizer, e)
    train_avg_loss_rec.append(sum(train_loss_rec) / len(train_loader))

    test_loss = test(net, test_loader, optimizer, e)
    test_avg_loss_rec.append(sum(test_loss_rec) / (len(test_loader)))

     #----------Save the model every 10 epochs
    if (e + 1) % 10 == 0:
       model_path = checkpoint_path + f"StdDev0_1_model_epoch_{e + 1}.pt"
       torch.save(net.state_dict(), model_path)

    # #---------------------------------------------------------- Access the out_en tensor
    out_en = net.out_en
    out_en_numpy = out_en.cpu().detach().numpy()
    # Save with a different name for each epoch
    out_en_filename = Output_Spikes + f"StdDev0_1_out_en_epoch_{e + 1}.npy"
    np.save(out_en_filename, out_en_numpy)

    # #-----------------------------------------------------------Access the out tensor
    out = net.out
    out_numpy = out.cpu().detach().numpy()
    # Save with a different name for each epoch
    out_filename = Output_Spikes + f"StdDev0_1_out_epoch_{e + 1}.npy"
    np.save(out_filename, out_numpy)
    # -------------------------------------------------------------------- Intermediate Layers
# Check if the current epoch is a multiple of 10
    if (e + 1) % 10 == 0:
        # Save the epoch_activations dictionary to a file
        activations_path = Intermediate_Lyrs +  f"StdDev0_1_activations_epoch_{e + 1}.pkl"
        # activations_path = Intermediate_Lyrs +  f"StdDev0_1_OverFrng_activations_epoch_{e + 1}.pkl"
        with open(activations_path, 'wb') as file:
            pickle.dump(epoch_activations, file)
    # Capture activations for the current epoch
    epoch_activations[e] = {}
    for i, name in enumerate(hook_names):
        epoch_activations[e][name] = activation.get(name, None)  # Use get to avoid KeyError




# beta_syn=0.9


Streaming output truncated to the last 5000 lines.
Train[30/50][16/240] Loss: 0.11156463623046875, BER : 0.04204296875
Train[30/50][17/240] Loss: 0.11268308758735657, BER : 0.042125
Train[30/50][18/240] Loss: 0.11138640344142914, BER : 0.04152734375
Train[30/50][19/240] Loss: 0.11234129965305328, BER : 0.04269140625
Train[30/50][20/240] Loss: 0.11419010162353516, BER : 0.0435703125
Train[30/50][21/240] Loss: 0.11199674755334854, BER : 0.04142578125
Train[30/50][22/240] Loss: 0.1090904250741005, BER : 0.04060546875
Train[30/50][23/240] Loss: 0.11163678020238876, BER : 0.04158203125
Train[30/50][24/240] Loss: 0.11199823766946793, BER : 0.04175390625
Train[30/50][25/240] Loss: 0.11562618613243103, BER : 0.04432421875
Train[30/50][26/240] Loss: 0.11235829442739487, BER : 0.04210546875
Train[30/50][27/240] Loss: 0.1141076534986496, BER : 0.0424921875
Train[30/50][28/240] Loss: 0.11394181102514267, BER : 0.04315625
Train[30/50][29/240] Loss: 0.11319425702095032, BER : 0.0430625
Train[30/50][

In [ ]:
# ////////////////////////////////////// just for printing elapsed time
def format_time(seconds):
    if seconds < 300:
        s = float(seconds)
        return "%.1f seconds" % (s,)
    elif seconds < 3600:
        m = seconds / 60.0
        return "%.2f minutes" % (m,)
    else:
        h = seconds / 3600.0
        return "%.2f hours" % (h,)

dt = datetime.datetime.now() - start_time
t = format_time(dt.total_seconds())

end_time = datetime.datetime.now()
dt = datetime.datetime.now() - start_time
seconds = dt.total_seconds()
t = format_time(seconds)
print("time: %s " % (t))
# /////////////////////////////////////////////////